https://programminghistorian.org/en/lessons/extracting-illustrated-pages

In [2]:
pip install internetarchive


The following command must be run outside of the IPython shell:

    $ pip install internetarchive

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [1]:
# Import the IA Python library
import internetarchive as ia

# Replace placeholder strings with your IA credentials (leaving the quote marks)
ia_email = "julia.knuchel@ub.unibe.ch"
ia_password = "knuchi93"

# add these credentials to the API's configuration object
ia.configure(ia_email, ia_password)

'/Users/juliaknuchel/.config/ia.ini'

In [2]:
# the requests library installed through conda
import requests

# a few other imports from the Python standard library
import gzip
import os
import sys
import xml.etree.ElementTree as ET

In [3]:
# sample search (should yield two results)
query = "peter parley date:[1825 TO 1830] mediatype:texts"
vol_ids = [result['identifier'] for result in ia.search_items(query)]
vol_ids

['UF00003119', 'talespeterparle00goodgoog']

In [14]:
# define a function for downloading pictures from a given IA volume
def ia_picture_download(item_id, out_dir= "/Users/juliaknuchel/Python/Hands-On/API/lesson-files"):
    """
    :param item_id: unique Internet Archive volume identifier
    :param out_dir: destination for images; if None, no download
    
    Note: if supplied, out_dir must be an existing directory and
    the caller must have write permissions in that directory
    
    :rtype list of pages with one or more blockType=Picture in Abbyy OCR data
    """

    print("[{}] Starting processing".format(item_id))
    
    # Use command-line client to see available metadata formats:
    # `ia metadata formats VOLUME_ID`
    
    # for this lesson, only the Abbyy file is needed
    returned_files = list(ia.get_files(item_id, formats=["Abbyy GZ"]))
    
    # make sure something got returned
    if len(returned_files) > 0:
        abbyy_file = returned_files[0].name
    else:
        print("[{}] Could not get Abbyy file".format(item_id))
        return None
    
    # download the abbyy file to CWD
    ia.download(item_id, formats=["Abbyy GZ"], ignore_existing=True, destdir=os.getcwd(), no_directory=True)
    
    # collect the pages with at least one picture block
    img_pages = []
    
    with gzip.open(abbyy_file) as fp:
        tree = ET.parse(fp)
        document = tree.getroot()
        for i, page in enumerate(document):
            for block in page:
                try:
                    if block.attrib['blockType'] == 'Picture':
                        img_pages.append(i)
                        break
                except KeyError:
                    continue
    
    # 0 is not a valid page for making GET requests to IA,
    #yet sometimes it's in the zipped Abbyy file
    img_pages = [page for page in img_pages if page > 0]
    
    # track for download progress report
    total_pages = len(img_pages)

    # OCR files are huge, so just delete once we have pagelist
    os.remove(abbyy_file)
    
    # if out_dir is not None, then also download page images
    if out_dir:
        
        # return if folder already exists (reasonable inference that volume already processed)
        if os.path.isdir(out_dir):
            print("[{}] Directory already exists.".format(item_id))
            return img_pages

        # otherwise, create folder to put the images
        print("[{}] Making directory {}".format(item_id, out_dir))
        os.makedirs(out_dir)
        
        # https://iiif.archivelab.org/iiif/documentation
        urls = ["https://iiif.archivelab.org/iiif/{}${}/full/full/0/default.jpg".format(item_id, page) for page in img_pages]
        
        # no direct page download through API, DIY
        for i, page, url in zip(range(1,total_pages), img_pages, urls):
            rsp = requests.get(url, allow_redirects=True)
            if rsp.status_code == 200:
                print("[{}] Downloading page {} ({}/{})".format(item_id, page, i+1, total_pages))
                with open(os.path.join(out_dir, str(page) + ".jpg"), "wb") as fp:
                    fp.write(rsp.content)
    
    # return list of pages with 1+ picture blocks
    return img_pages

In [6]:
# loop over our search results and call the function
for item_id in vol_ids:
    destination = os.path.join("items", "internetarchive", item_id)
    img_pages = ia_picture_download(item_id, out_dir=destination)

[UF00003119] Starting processing
[UF00003119] Could not get Abbyy file
[talespeterparle00goodgoog] Starting processing
talespeterparle00goodgoog: d - success
[talespeterparle00goodgoog] Making directory items/internetarchive/talespeterparle00goodgoog
[talespeterparle00goodgoog] Downloading page 1 (2/43)
[talespeterparle00goodgoog] Downloading page 2 (3/43)
[talespeterparle00goodgoog] Downloading page 7 (4/43)
[talespeterparle00goodgoog] Downloading page 8 (5/43)
[talespeterparle00goodgoog] Downloading page 12 (6/43)
[talespeterparle00goodgoog] Downloading page 14 (7/43)
[talespeterparle00goodgoog] Downloading page 16 (8/43)
[talespeterparle00goodgoog] Downloading page 17 (9/43)
[talespeterparle00goodgoog] Downloading page 18 (10/43)
[talespeterparle00goodgoog] Downloading page 19 (11/43)
[talespeterparle00goodgoog] Downloading page 22 (12/43)
[talespeterparle00goodgoog] Downloading page 27 (13/43)
[talespeterparle00goodgoog] Downloading page 33 (14/43)
[talespeterparle00goodgoog] Downl

In [7]:
# 2. Beispiel
# sample search (should yield two results)
query = "knuchel date:[1800 TO 2000] mediatype:texts"
vol_ids = [result['identifier'] for result in ia.search_items(query)]
vol_ids

['dieumwandlungink00knucuoft']

In [9]:
# 2. beispiel
# loop over our search results and call the function
for item_id in vol_ids:
    destination = os.path.join("items", "internetarchive", item_id)
    img_pages = ia_picture_download(item_id, out_dir=destination)

[dieumwandlungink00knucuoft] Starting processing
dieumwandlungink00knucuoft: d - success
[dieumwandlungink00knucuoft] Making directory items/internetarchive/dieumwandlungink00knucuoft
[dieumwandlungink00knucuoft] Downloading page 85 (2/3)
[dieumwandlungink00knucuoft] Downloading page 86 (3/3)


In [13]:
# 3. Beispiel Suche nach dem Wort Gottesdienst zwischen 1800 und 1850, wird in vol_ids gespeichert
# sample search
query = "gottesdienst date:[1800 TO 1850]"
vol_ids = [result['identifier'] for result in ia.search_items(query)]
vol_ids

['bub_gb_-C1FAAAAcAAJ',
 'bub_gb_3As3AAAAMAAJ',
 'bub_gb_6jpFAAAAcAAJ',
 'bub_gb_i0H99K47LuwC',
 'dasneueundverbes00bill',
 'dersegendesgotte00hold',
 'diepsalmenundfes00lobw',
 'einesammlungvong00rain_0',
 'katholischesges00prgoog']

In [ ]:
# 3. beispiel
# loop over our search results and call the function
# was spuckt es jetzt genau aus? alle Bilder aus allen den gefundenen Dokumenten? 
# Sind aber nicht alles Bilder, ev. Text der nicht OCR ist und deshalb als Bild erkannt wird?
for item_id in vol_ids:
    destination = os.path.join("items", "internetarchive", item_id)
    img_pages = ia_picture_download(item_id, out_dir=destination)

In [22]:
# 4. Beispiel Suche nach Werken von Jane Austen, die zu ihrer Lebzeit publiziert wurden (1775-1817)
# sample search
query = "creator:Jane Austen date:[1780 TO 1900] mediatype:texts language:english" 
vol_ids = [result['identifier'] for result in ia.search_items(query)]
vol_ids

['LadySusan',
 'amemoirjaneaust01austgoog',
 'amemoirjaneaust02austgoog',
 'bub_gb_Im4JAAAAQAAJ',
 'bub_gb_TJYMUGRIUTMC',
 'chaptersfromjan00austgoog',
 'chaptersfromjan01austgoog',
 'duologuesandsce00filigoog',
 'emma00austgoog',
 'emma01austgoog',
 'emma02austgoog',
 'emmaanovel00austgoog',
 'emmaanovelinthr00austgoog',
 'emmaanovelinthr00unkngoog',
 'emmaanovelinthr01austgoog',
 'emmaanovelinthr02austgoog',
 'emmaanovelinthr03austgoog',
 'emmabyjaneauste00austgoog',
 'emmabyjaneauste01austgoog',
 'ladysusanwatson00austgoog',
 'ladysusanwatson01austgoog',
 'ladysusanwatson02austgoog',
 'ladysusanwatson03austgoog',
 'lettersjaneaust00austgoog',
 'lettersjaneaust00coolgoog',
 'lettersjaneaust01austgoog',
 'lettersjaneaust01coolgoog',
 'mansfieldpark02austgoog',
 'mansfieldpark03austgoog',
 'mansfieldpark05austgoog',
 'mansfieldpark07austgoog',
 'mansfieldpark08austgoog',
 'mansfieldpark09austgoog',
 'mansfieldpark10austgoog',
 'mansfieldpark11austgoog',
 'mansfieldparkan00austgoog',
 '